# The potential model, custom loop

In [1]:
import tensorflow as tf
import tensorflow as tf
from glob import glob
from pinn.io import load_qm9
from docs.notebooks.network_fns import get_traintest_sets, preprocess_traintest_sets
from pinn.layers import PolynomialBasis, GaussianBasis, ANNOutput
from pinn.networks.pinet import OutLayer, GCBlock, ResUpdate, PreprocessLayer

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = load_qm9(filelist, splits={'train':8, 'test':2})

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-26 18:22:22.270766: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-26 18:22:22.270846: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
params={
    'optimizer': {
        'class_name': 'Adam',
        'config': {
            'learning_rate': {
                'class_name': 'ExponentialDecay',
                'config': {
                    'initial_learning_rate': 0.0003,
                    'decay_steps': 10000, 
                    'decay_rate': 0.994}}, 
                    'clipnorm': 0.01}}
}

In [4]:
class PiNet(tf.keras.Model):
    """Keras model for the PiNet neural network

    Args:
        tensors: input data (nested tensor from dataset).
        atom_types (list): elements for the one-hot embedding.
        pp_nodes (list): number of nodes for pp layer.
        pi_nodes (list): number of nodes for pi layer.
        ii_nodes (list): number of nodes for ii layer.
        en_nodes (list): number of nodes for en layer.
        depth (int): number of interaction blocks.
        rc (float): cutoff radius.
        basis_type (string): type of basis function to use,
            can be "polynomial" or "gaussian".
        n_basis (int): number of basis functions to use.
        gamma (float or array): width of gaussian function for gaussian basis.
        center (float or array): center of gaussian function for gaussian basis.
        cutoff_type (string): cutoff function to use with the basis.
        act (string): activation function to use.
        preprocess (bool): whether to return the preprocessed tensor.
    """
    def __init__(self, atom_types=[1, 6, 7, 8, 9],  rc=4.0, cutoff_type='f1',
                 basis_type='polynomial', n_basis=4, gamma=3.0, center=None,
                 pp_nodes=[16, 16], pi_nodes=[16, 16], ii_nodes=[16, 16],
                 out_nodes=[16, 16], out_units=1, out_pool=False,
                 act='tanh', depth=4):

        super(PiNet, self).__init__()

        self.depth = depth
        self.preprocess = PreprocessLayer(atom_types, rc)
        self.activation = act

        if basis_type == 'polynomial':
            self.basis_fn = PolynomialBasis(cutoff_type, rc, n_basis)
        elif basis_type == 'gaussian':
            self.basis_fn = GaussianBasis(cutoff_type, rc, n_basis, gamma, center)

        self.res_update = [ResUpdate() for i in range(depth)]
        self.gc_blocks = [GCBlock([], pi_nodes, ii_nodes, activation=act)]
        self.gc_blocks += [GCBlock(pp_nodes, pi_nodes, ii_nodes, activation=act)
                           for i in range(depth-1)]
        self.out_layers = [OutLayer(out_nodes, out_units) for i in range(depth)]
        self.ann_output =  ANNOutput(out_pool)
    
    def call(self, tensors):
        tensors = self.preprocess(tensors)
        basis = self.basis_fn(tensors['dist'])[:, None, :]

        output = 0.0
        for i in range(self.depth):
            prop = self.gc_blocks[i]([tensors['ind_2'], tensors['prop'], basis])
            output = self.out_layers[i]([tensors['ind_1'], prop, output])
            tensors['prop'] = self.res_update[i]([tensors['prop'], prop])

        output = self.ann_output([tensors['ind_1'], output])
        return output

In [5]:
network = PiNet()
train_set, test_set, batch_size = get_traintest_sets(dataset=dataset, buffer_size=20000, batch_size=256)
preprocess_traintest_sets(train_set, test_set, network=network)

2023-03-26 18:22:28.666551: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-03-26 18:22:28.666776: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [6]:
train_err_metric = tf.keras.metrics.MeanAbsoluteError()
val_err_metric = tf.keras.metrics.MeanAbsoluteError()

In [7]:
#Instantiate an optimizer
import time
from pinn.optimizers import get
optimizer = get(params['optimizer'])
loss_fn = tf.keras.losses.mse

epochs = 1
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch}")
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(train_set):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            pred = network(batch, training=True)  # Logits for this minibatch

            ind = batch['ind_1']
            nbatch = tf.reduce_max(ind)+1
            pred = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)


            # Compute the loss value for this minibatch.
            loss_value = loss_fn(batch['e_data'], pred)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, network.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, network.trainable_weights))

        # #Update the training metric
        # train_err_metric.update_state(batch['e_data'], pred)

        # Log every 100 batches.
        if step % 100 == 0:
            print(
                f"Training loss (for one batch) at step {step}: {float(loss_value)}"
            )
            print(f"Seen so far: {((step + 1) * batch_size)} molecules")
    
    #Update the training metric now that the network has been trained
    for batch in train_set:
        pred = network(batch, training=False)  # Logits for this minibatch

        ind = batch['ind_1']
        nbatch = tf.reduce_max(ind)+1
        pred = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
        train_err_metric.update_state(batch['e_data'], pred)
    
    #Display metrics at the end of each epoch
    train_err = train_err_metric.result()
    print(f"Training err over epoch: {float(train_err)}")

    #Reset training metrics at the end of each epoch
    train_err_metric.reset_states()

    #Run a validation loop at the end of each epoch
    for batch in test_set:
        val_pred = network(batch, training=False)
        ind = batch['ind_1']
        nbatch = tf.reduce_max(ind)+1
        val_pred = tf.math.unsorted_segment_sum(val_pred, ind[:, 0], nbatch)

        #Update val metrics
        val_err_metric.update_state(batch['e_data'], val_pred)
        
    val_err = val_err_metric.result()
    val_err_metric.reset_states()
    print(f"Validation err: {float(val_err)}")
    print(f"Time taken: {(time.time() - start_time)}")


Start of epoch 0
Training loss (for one batch) at step 0: 200119.703125
Seen so far: 256 molecules
Training loss (for one batch) at step 100: 1675.4761962890625
Seen so far: 25856 molecules
Training loss (for one batch) at step 200: 959.65966796875
Seen so far: 51456 molecules
Training loss (for one batch) at step 300: 694.817626953125
Seen so far: 77056 molecules
Training loss (for one batch) at step 400: 599.907470703125
Seen so far: 102656 molecules
Training err over epoch: 23.2858943939209
Validation err: 23.117355346679688
Time taken: 513.5562260150909
